# ex3-Overlay Raster on Interactive Map with Your Own Tile Server 

In the [previous tutorial](https://www.linkedin.com/pulse/overlay-raster-nodata-interactive-map-folium-chonghua-yin/?lipi=urn%3Ali%3Apage%3Ad_flagship3_profile_view_base_post_details%3BGepBWn%2BWTCuIK2DsSiPCgw%3D%3D), we already went through how to create interactive maps with Python in Jupyter Notebook using the folium package and how to overlay a raster on an interactive map created with Folium, even when the raster came with nodata values. [***folium***](https://python-visualization.github.io/folium/) friendly provides the following built-in tilesets as the basemaps that we could access to create our interactive maps:

- OpenStreetMap
- Mapbox Bright (Limited levels of zoom for free tiles)
- Mapbox Control Room (Limited levels of zoom for free tiles)
- Stamen (Terrain, Toner, and Watercolor)
- Cloudmade (Must pass API key)
- Mapbox (Must pass API key)
- CartoDB (positron and dark_matter).

However, you should need to know that these basemaps come from APIs that are provided by various organizations such as OpenStreetMap, MapBox, Stamen, Google, etc. Although tiles from a third-party provider are the simplest way to make basemaps,  you may still want to render and serve your own tiles for some reasons (e.g., you’d like to take full control of your map or because Google has raised the prices of Google Maps APIs). It is totally possible in Folium and you just need provide a custom tileset URL. 

In this tutorial, let's set up an OpenStreetMap tile server and then overlay a raster on it.

## 1. Set up OpenStreetMap tile server

OpenStreetMap, aka OSM, is a user-contributed world map that is freely editable. You can think of it as an open-source and self-hosted alternative to Google Maps. 

[OpenStreetMap](https://www.openstreetmap.org/about) has several key features:

- OpenStreetMap data covers the whole world, making it easy to support users in any country, or every country.
- OpenStreetMap is updated every minute of every hour of every day, and these updates are available to you in real-time.
- OpenStreetMap data is free and open – there is no subscription fee and no page-view fee.
- OpenStreetMap data is rich and detailed, containing huge amounts of data which is relevant to people on the ground – the people who collected it.

This section will show you how to easily set up an OpenStreetMap PNG tile server given a .osm.pbf file using Docker, so you don’t have to use a proprietary map service.

## 1.1 Prepare OpenStreetMap data

You can download data from the [Geofabrik's free download server](http://download.geofabrik.de/). The server has data extracts from the OpenStreetMap project which are normally updated every day. You can select your continent and then your country of interest from the list on the webpage. Here we download the data for [Australia and Oceania]( http://download.geofabrik.de/australia-oceania-latest.osm.pbf).

In [ ]:
wget http://download.geofabrik.de/australia-oceania-latest.osm.pbf

## 1.2 Set up server

Here we use the [docker image provide by overv](https://hub.docker.com/r/overv/openstreetmap-tile-server/) to setup our OpenStreetMap tile server. The image was created based on the latest Ubuntu 18.04 LTS guide from switch2osm.org and used the default OpenStreetMap style. 

Shortly, the following tools are used for generating and serving map tiles.

- Apache 

provides the front end server that handles requests from your web browser and passes the request to mod_tile. 

- mod_tile

Apache passes the request from a web user to mod_tile. Mod_tile checks if the tile has already been created and is ready for use or whether it needs to be updated due to not being in the cache already. If it is already available and doesn’t need to be rendered, then it immediately sends the tile back to the client. If it does need to be rendered, then it will add it to a “render request” queue, and when it gets to the top of the queue, a tile renderer will render it and send the tile back to the client.

- Mapnik

Mapnik is used to render tiles. It pulls requests from the work queue as fast as possible, extracts data from various data sources according to the style information, and renders the tile. This tile is passed back to the client and moves on to the next item in the queue.

- PostgreSQL, PostGIS and osm2pgsql

osm2pgsql is used to import OpenStreetMap data into a PostgreSQL database for rendering by Mapnik, while “postgis” adds some extra graphical supports PostgreSQL. 

For each of them, you can check the Dockerfile for details.

#### Step 1: create a Docker volume to hold the PostgreSQL database that will contain the OpenStreetMap data

In [ ]:
docker volume create openstreetmap-data

#### Step 2: create a Docker volume to preserve rendered tiles

Tiles that have already been rendered will be stored in /var/lib/mod_tile. To make sure that this data survives container restarts, you should create another volume for it.

In [ ]:
docker volume create openstreetmap-rendered-tiles

#### Step 3: import the OpenStreetMap data of australia-oceania-latest.osm.pbf into PostgreSQL

In [ ]:
docker run \
    -v pathto\australia-oceania-latest.osm.pbf:/data.osm.pbf \
    -v openstreetmap-data:/var/lib/postgresql/10/main \
    overv/openstreetmap-tile-server import

#### Step 4: start the server

In [ ]:
docker run -p 80:80 \
    -v openstreetmap-data:/var/lib/postgresql/10/main \
    -v openstreetmap-rendered-tiles:/var/lib/mod_tile \
    overv/openstreetmap-tile-server run

If everything works well, you can check the tiles by inputting the http://localhost/tile/1/1/1.png into your browser. These tiles will be served as the format of http://localhost:80/tile/{z}/{x}/{y}.png, which can be used in our interactive map created by folium in the following section. It is worth noting that it will take initially quite a bit of time to render the larger tiles for the first time.

In addition, you only need  use the ***step 4*** to start the tile server every time when you want to play with it or you can let it run in the background with an extra docker run option ***-d***.

In [ ]:
docker run -p 80:80 \
    -v openstreetmap-data:/var/lib/postgresql/10/main \
    -v openstreetmap-rendered-tiles:/var/lib/mod_tile \
    -d \
    overv/openstreetmap-tile-server run

## 2. Overlay Raster

This section just copied the content from the previous tutorial. You can jump to the ***section of 2.4*** to see the final map. 
:) please check my own unique mark of ***chy@climsystems*** at the right bottom of the overlaid map to verify that we are using custom tiles.

### 2.1 Load libs

In [1]:
%matplotlib inline

import numpy as np
import folium
import branca.colormap as cm

import rasterio as rio
import matplotlib.pyplot as plt
from matplotlib import colors as colors

### 2.2 Prepare data

The demo tif is downloaded from https://github.com/stuartmatthews/stuartmatthews.github.io/tree/master/leaflet-geotiff/tif. The package of rasterio is used to read the tif data and get other related information. As we know, wind speed must be greater than 0.0 and so we set the values with data<0.0 to nans. We also need boundary information to calculate its center.

In [2]:
infile = "../data/wind_speed.tif"
with rio.open(infile) as src:
    boundary = src.bounds
    img = src.read()
    nodata = src.nodata    

img[img<0.0] = np.nan

clat = (boundary.bottom + boundary.top)/2
clon = (boundary.left + boundary.right)/2

### 2.3 Define colormap and color mapping function

Here we use only one built-in linear colormap of branca.colormaps as an example. Maybe it is not the best one. However, you can check the whole set of bolium.colormap.linear by inputting and running ***cm.linear*** in a cell. We use the minimum (0.0) and maximum (30.0) wind speed to scale the colors of ***branca.colormap.linear.RdBu_11***. They can be calculated directly from the data array of ***img***.

In [3]:
vmin = np.floor(np.nanmin(img))
vmax = np.ceil(np.nanmax(img))

colormap = cm.linear.RdBu_11.scale(vmin, vmax)

***color mapping function*** to fight nodata

This color mapping function will filters nans and map a normal value to a color. Finally, return a color value from the ***hex*** to the ***rgba*** format that can be used by the [colormap of Folium Raster Layers](https://python-visualization.github.io/folium/modules.html#module-folium.raster_layers).

In [4]:
def mapvalue2color(value, cmap): 
    """
    Map a pixel value of image to a color in the rgba format. 
    As a special case, nans will be mapped totally transparent.
    
    Inputs
        -- value - pixel value of image, could be np.nan
        -- cmap - a linear colormap from branca.colormap.linear
    Output
        -- a color value in the rgba format (r, g, b, a)    
    """
    if np.isnan(value):
        return (1, 0, 0, 0)
    else:
        return colors.to_rgba(cmap(value), 0.7)  

### 2.4 Overlay Raster

We use the ***Map()*** function from folium and the calculated latitude (***clat***) and longitude(***clon***) to center our map.

There are a number of built-in tilesets from OpenStreetMap, Mapbox, and Stamen. Here the map is created using the default basemap from ***OpenStreetMap***. 

To overlay data on web-based basemaps, the overlay data needs to be in the ***WGS84 coordinate system***. Good luck, our raster is right in the correct coordinate system (WGS84). Now we can overlay it on the basemp using the folium.raster_layers.***ImageOverlay()*** and the ***add_child()*** function.

In addition, the colormap (i.e., from ***branca.colormap.linear.RdBu_11***) we defined above is also a folium element that we can also draw on the map.

In [7]:
m = folium.Map(location=[clat, clon], 
               tiles='http://localhost:80/tile/{z}/{x}/{y}.png', attr="OpenStreetMap, Tiled by chy@climsystems",
               zoom_start=5)

folium.raster_layers.ImageOverlay(
    image=img[0],
    name='Wind Speed Map',
    opacity=1,
    bounds= [[boundary.bottom, boundary.left], [boundary.top, boundary.right]],
    colormap= lambda value: mapvalue2color(value, colormap)
).add_to(m)

folium.LayerControl().add_to(m)
colormap.caption = 'Wind Speed'
m.add_child(colormap)
m

### 2.5 Bonus

In addition, you can save the map to an HTML file just like the following command. Then you can play it with your favorite web browsers.

In [6]:
m.save('WSP_NSW_AUS.html')

## 3. A little bit more

In fact, deploying a custom OpenStreetMap tile server is not only beneficial for creating inactive maps with folium just as we did in this tutorial, but may also helpful for your other applications as it is free of charge and you do not have to depend on other tileset providers. 

There are quite a few online tutorials about setting up a a custom tile server based on open-source tools and data such as  PostGIS and OpenStreetMap data. Moreover, with PostgreSQL 12 and PostGIS 3, most spatial queries that can take advantage of parallel processing would get more aggressively. It is worth a try!

If you are running Ubuntu 18.04, I strongly recommend you to have a look at the great step-by-step tutorial of "[How to Set Up OpenStreetMap Tile Server on Ubuntu 18.04](https://www.linuxbabe.com/ubuntu/openstreetmap-tile-server-ubuntu-18-04-osm)".

## References and Resources

https://python-visualization.github.io/folium/modules.html#module-folium.raster_layers

https://rasterio.readthedocs.io/en/stable/index.html

https://www.linuxbabe.com/ubuntu/openstreetmap-tile-server-ubuntu-18-04-osm

https://hub.docker.com/r/overv/openstreetmap-tile-server/

https://github.com/zavpyj/osm-tiles-docker